In [3]:
import re
import numpy as np
import pandas as pd
from pprint import pprint

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# spacy for lemmatization
import spacy

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)


usaid = pd.read_csv('/Users/vibhuverma/Desktop/CLASSWORK/BUSINESS PRACTICUM/Anti-Corruption/Data Provided/USAID_Project_Data/USAID_Anticorruption_Projects_Database.csv')

usaid.drop(['proj_res','rfp_rftop','final_res','mind_eval','final_eval','audit','interim_rep','addl_docs','addl_info'],axis=1,inplace=True)

df = usaid[['proj_desc']]



In [4]:
# Convert to list
data = df.proj_desc.values.tolist()

# Remove Emails
data = [re.sub('\S*@\S*\s?', '', sent) for sent in data]

# Remove new line characters
data = [re.sub('\s+', ' ', sent) for sent in data]

# Remove distracting single quotes
data = [re.sub("\'", "", sent) for sent in data]

pprint(data[:1])

['The five components of this program seek to take what already exists and '
 'improve upon it, implement it and find ways to ensure that the progress '
 'achieved is sustainable over the long term. The project will help the '
 'government of the Islamic Republic of Afghanistan strengthen its High Office '
 'of Oversight (HOO) - making it a strong, effective institution that is able '
 'to lead, monitor, coordinate and report on efforts to combat corruption '
 'across the country. The project will work and implement a program of '
 'institutional development and sustainability. It will also support the '
 'office in carrying out its priority responsibilities in asset registration '
 'and verification, complaints management and case tracking, and coordination '
 'and monitoring of anticorruption performance across other government '
 'agencies.']


In [5]:
# NLTK Stop words
from nltk.corpus import stopwords
stop_words = stopwords.words('english')

In [6]:
#Tokenize each word

def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

data_words = list(sent_to_words(data))

print(data_words[:1])

[['the', 'five', 'components', 'of', 'this', 'program', 'seek', 'to', 'take', 'what', 'already', 'exists', 'and', 'improve', 'upon', 'it', 'implement', 'it', 'and', 'find', 'ways', 'to', 'ensure', 'that', 'the', 'progress', 'achieved', 'is', 'sustainable', 'over', 'the', 'long', 'term', 'the', 'project', 'will', 'help', 'the', 'government', 'of', 'the', 'islamic', 'republic', 'of', 'afghanistan', 'strengthen', 'its', 'high', 'office', 'of', 'oversight', 'hoo', 'making', 'it', 'strong', 'effective', 'institution', 'that', 'is', 'able', 'to', 'lead', 'monitor', 'coordinate', 'and', 'report', 'on', 'efforts', 'to', 'combat', 'corruption', 'across', 'the', 'country', 'the', 'project', 'will', 'work', 'and', 'implement', 'program', 'of', 'institutional', 'development', 'and', 'sustainability', 'it', 'will', 'also', 'support', 'the', 'office', 'in', 'carrying', 'out', 'its', 'priority', 'in', 'asset', 'registration', 'and', 'verification', 'complaints', 'management', 'and', 'case', 'tracking

In [7]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

# See trigram example
print(trigram_mod[bigram_mod[data_words[0]]])

['the', 'five', 'components', 'of', 'this', 'program', 'seek', 'to', 'take', 'what', 'already', 'exists', 'and', 'improve', 'upon', 'it', 'implement', 'it', 'and', 'find', 'ways', 'to', 'ensure', 'that', 'the', 'progress', 'achieved', 'is', 'sustainable', 'over', 'the', 'long_term', 'the', 'project', 'will', 'help', 'the', 'government', 'of', 'the', 'islamic_republic', 'of', 'afghanistan', 'strengthen', 'its', 'high', 'office', 'of', 'oversight', 'hoo', 'making', 'it', 'strong', 'effective', 'institution', 'that', 'is', 'able', 'to', 'lead', 'monitor', 'coordinate', 'and', 'report', 'on', 'efforts', 'to', 'combat', 'corruption', 'across', 'the', 'country', 'the', 'project', 'will', 'work', 'and', 'implement', 'program', 'of', 'institutional', 'development', 'and', 'sustainability', 'it', 'will', 'also', 'support', 'the', 'office', 'in', 'carrying', 'out', 'its', 'priority', 'in', 'asset', 'registration', 'and', 'verification', 'complaints', 'management', 'and', 'case', 'tracking', 'and

In [8]:
# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [9]:
# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# python3 -m spacy download en
nlp = spacy.load('en', disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(data_lemmatized[:1])

[['component', 'program', 'seek', 'take', 'already', 'exist', 'improve', 'implement', 'find', 'way', 'ensure', 'progress', 'achieve', 'sustainable', 'project', 'help', 'government', 'islamic_republic', 'strengthen', 'high', 'office', 'oversight', 'hoo', 'make', 'strong', 'effective', 'institution', 'able', 'lead', 'monitor', 'coordinate', 'report', 'effort', 'combat', 'corruption', 'country', 'project', 'work', 'implement', 'program', 'institutional', 'development', 'sustainability', 'also', 'support', 'office', 'carry', 'priority', 'asset', 'registration', 'verification', 'complaint', 'management', 'case', 'track', 'coordination', 'monitor', 'anticorruption', 'performance', 'government', 'agency']]
